# First Spark Streaming Example
_____

# Twitter Example
Set up the credentials for a twitter app at https://apps.twitter.com/
    
install python-twitter, a python library to connect your Python to the twitter dev account.

Begin by running the TweetRead.py file. Make sure to add your own IP Adress and your credential keys.

In [ ]:
!pip install findspark

In [1]:
import findspark

In [2]:
# your path will likely not have 'matthew' in it. Change it to reflect your path.
findspark.init('/home/polo/Workspace/Self/DataSience/Spark/spark-3.1.2-bin-hadoop2.7')

In [3]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [4]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

In [5]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [6]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

In [7]:
lines = socket_stream.window( 20 )

In [8]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [9]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print('------------------------ %s ----------------------' % str(time))
    sql_context = get_sql_context_instance(rdd.context)
    row_rdd = rdd.map(lambda rec: Tweet( rec[0], rec[1] ))
    hashtags_df = sql_context.createDataFrame(row_rdd)
    hashtags_df.registerTempTable('tweets')
    hashtags_df.show()
    print('------------------------------OK----------------------------')

In [10]:
# Use Parenthesis for multiple lines or use \.
 #Splits to a list
words = lines.flatMap( lambda text: text.split( " " ) )
 # Checks for hashtag calls
hastags = words.filter( lambda word: word.lower().startswith("#") )\
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word

tags_total = hastags.updateStateByKey( lambda a, b: a + b ) # Reduces
tags_total.foreachRDD(process_rdd)
# .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
#   #.foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
#   .foreachRDD( lambda rdd: rdd.toDF().sort("count")
#   .limit(10).registerTempTable("tweets") ))  # Registers to a table.

__________
### Run the TweetRead.py file at this point
__________

In [11]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [12]:
ssc.start()

IllegalArgumentException: requirement failed: The checkpoint directory has not been set. Please set it by StreamingContext.checkpoint().

In [ ]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    sql_context = get_sql_context_instance(sqlContext)
    top_10_tweets = sql_context.sql( 'Select tag, count from tweets limit 10' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    plt.show()
    count = count + 1

In [ ]:
ssc.stop()